In [2]:
import numpy as np
from scipy.linalg import sqrtm 
from scipy.special import softmax
import networkx as nx
from networkx.algorithms.community.modularity_max import greedy_modularity_communities
import matplotlib.pyplot as plt
from matplotlib import animation
%matplotlib inline


# Message Passing as Matrix Multiplication

In [6]:
#we have 5 nodes ,A is adjacency matrix
A = np.array(
    [[0, 1, 0, 0, 0], [1, 0, 1, 0, 0], [0, 1, 0, 1, 1], [0, 0, 1, 0, 0], [0, 0, 1, 0, 0]]
)
#Lets take a feature 
X = np.arange(A.shape[0]).reshape((-1,1))+1

In [7]:
print(A)
print(X)

[[0 1 0 0 0]
 [1 0 1 0 0]
 [0 1 0 1 1]
 [0 0 1 0 0]
 [0 0 1 0 0]]
[[1]
 [2]
 [3]
 [4]
 [5]]


In [8]:
#calculating 1 hop feature update, by simple matrix multiplication
#We can see the output of H, some of the features(for the node who have higher degree) after aggregation, tends to increase
H = A @ feats
H

array([[ 2],
       [ 4],
       [11],
       [ 3],
       [ 3]])

## Scale neighborhood sum by neighborhood size (i.e. average values)

In [9]:
#Diagonal matrix using Adjacency matrix(row sum)
D = np.zeros(A.shape)
np.fill_diagonal(D, A.sum(axis=0))
D

array([[1., 0., 0., 0., 0.],
       [0., 2., 0., 0., 0.],
       [0., 0., 3., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.]])

In [10]:
D_inv = np.linalg.inv(D)
D_inv

array([[1.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.5       , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.33333333, 0.        , 0.        ],
       [0.        , 0.        , 0.        , 1.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 1.        ]])

In [11]:
D_inv @ A 

array([[0.        , 1.        , 0.        , 0.        , 0.        ],
       [0.5       , 0.        , 0.5       , 0.        , 0.        ],
       [0.        , 0.33333333, 0.        , 0.33333333, 0.33333333],
       [0.        , 0.        , 1.        , 0.        , 0.        ],
       [0.        , 0.        , 1.        , 0.        , 0.        ]])

In [12]:
#message passing after normalizing adjacency matrix using diagonal matrix
H_avg = D_inv @ A @ feats
H_avg

array([[2.        ],
       [2.        ],
       [3.66666667],
       [3.        ],
       [3.        ]])

## Normalized Adjacency Matrix
Ultimately want to define and build:

$$ \hat{A} = \tilde{D}^{-\frac{1}{2}} \tilde{A} \tilde{D}^{-\frac{1}{2}} $$


First, create $\tilde{A}$:
$$ \tilde{A} = A + I $$

In [13]:
g = nx.from_numpy_array(A)
#making self loop, so that in aggregation part, i=the node will considers it't own previous features.
A_mod = A + np.eye(g.number_of_nodes())

In [14]:
# D for A_mod:
D_mod = np.zeros_like(A_mod)
np.fill_diagonal(D_mod, A_mod.sum(axis=1).flatten())

# Inverse square root of D:
D_mod_invroot = np.linalg.inv(sqrtm(D_mod))

In [15]:
D_mod

array([[2., 0., 0., 0., 0.],
       [0., 3., 0., 0., 0.],
       [0., 0., 4., 0., 0.],
       [0., 0., 0., 2., 0.],
       [0., 0., 0., 0., 2.]])

In [16]:
D_mod_invroot

array([[0.70710678, 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.57735027, 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.5       , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.70710678, 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.70710678]])

Create $\hat{A}$:

$$ \hat{A} = \tilde{D}^{-\frac{1}{2}} \tilde{A} \tilde{D}^{-\frac{1}{2}} $$

$$ (\hat{A})_{i,j} = \frac{\tilde{A}_{i,j}}{\sqrt{\tilde{d_i} \tilde{d_j}}} $$

In [23]:
#message passing (spectral)
A_hat = D_mod_invroot @ A_mod @ D_mod_invroot
A_hat

array([[0.5       , 0.40824829, 0.        , 0.        , 0.        ],
       [0.40824829, 0.33333333, 0.28867513, 0.        , 0.        ],
       [0.        , 0.28867513, 0.25      , 0.35355339, 0.35355339],
       [0.        , 0.        , 0.35355339, 0.5       , 0.        ],
       [0.        , 0.        , 0.35355339, 0.        , 0.5       ]])

### message passing in 10 iterations

In [26]:
H = np.zeros((g.number_of_nodes(), 1))
print(H)
H[0,0] = 1 # the "water drop"
iters = 10
results = [H.flatten()]
print(results)
for i in range(iters):
    H = A_hat @ H
    results.append(H.flatten())
print(results)

[[0.]
 [0.]
 [0.]
 [0.]
 [0.]]
[array([1., 0., 0., 0., 0.])]
[array([1., 0., 0., 0., 0.]), array([0.5       , 0.40824829, 0.        , 0.        , 0.        ]), array([0.41666667, 0.34020691, 0.11785113, 0.        , 0.        ]), array([0.34722222, 0.31752645, 0.12767206, 0.04166667, 0.04166667]), array([0.30324074, 0.28445078, 0.15304279, 0.06597222, 0.06597222]), array([0.26774691, 0.26279409, 0.16702397, 0.08709491, 0.08709491]), array([0.24115869, 0.24512092, 0.17920351, 0.10259934, 0.10259934]), array([0.22064954, 0.23189119, 0.18810988, 0.11465768, 0.11465768]), array([0.20499395, 0.22167951, 0.19504392, 0.12383573, 0.12383573]), array([0.19299726, 0.21388593, 0.20031942, 0.1308763 , 0.1308763 ]), array([0.18381719, 0.20791335, 0.20436693, 0.13626176, 0.13626176])]


In [27]:
print(f"Initial signal input: {results[0]}")
print(f"Final signal output after running {iters} steps of message-passing:  {results[-1]}")

Initial signal input: [1. 0. 0. 0. 0.]
Final signal output after running 10 steps of message-passing:  [0.18381719 0.20791335 0.20436693 0.13626176 0.13626176]
